<a id="table_of_content"></a>

# Part 1: Bag-of-Words Exercises

In the following, we will convert our corpus of 10-k documents into bag-of-words, and convert them into document vectors using Term-Frequency-Inverse-Document-Frequency (tf-idf) re-weighting. Afterward, we will compute sentiments and similarity metrics. Since we will be reusing our notebook, so the various sections are linked below:

1. <a href="#bag_of_word">Compute bag-of-words </a>: implement `bag_of_words` that converts tokenized words into a dictionary of word-counts

2. <a href="#sentiment">Sentiments </a>: using wordlists, compute positive and negative sentiments from bag-of-words. Implement `get_sentiment`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`

# Part 2: Document-Vector Exercises

3. <a href="#compute_idf">Compute idf </a>: computing the inverse document frequency, implement `get_idf`

4. <a href="#compute_tf">Compute tf </a>: computing the term frequency, implement `get_tf`

5. <a href="#doc_vector">Document vector </a>: using the functions `get_idf` and `get_tf`, compute a word_vector for each document in the corpus
6. <a href="#similarity">Similarities </a>: comparing two vectors, and compute cosine and jacard similarity metrics. Implement `get_cos` and `get_jac`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`


## 0. Initialization

First we read in our 10-k documents

In [1]:
# download some excerpts from 10-K files
from download10k import get_files

CIK = {'ebay': '0001065088', 'apple':'0000320193', 'sears': '0001310067'}
get_files(CIK['ebay'], 'EBAY')
get_files(CIK['apple'], 'AAPL')
get_files(CIK['sears'], 'SHLDQ')


# get a list of all 10-ks in our directory
files=! ls *10k*.txt
print("10-k files: ",files)
files = [open(f,"r").read() for f in files]

downloading 10-Ks item 1A for CIK = 0001065088 ...
downloading 10-Ks item 1A for CIK = 0000320193 ...
downloading 10-Ks item 1A for CIK = 0001310067 ...
10-k files:  ['AAPL_10k_2015.txt', 'AAPL_10k_2016.txt', 'AAPL_10k_2017.txt', 'AAPL_10k_2020.txt', 'EBAY_10k_2015.txt', 'EBAY_10k_2016.txt', 'EBAY_10k_2017.txt', 'SHLDQ_10k_2014.txt', 'SHLDQ_10k_2015.txt', 'SHLDQ_10k_2016.txt', 'SHLDQ_10k_2017.txt']


here we define useful functions to tokenize the texts into words, remove stop-words, and lemmatize and stem our words

In [2]:
import numpy as np

# for nice number printing
np.set_printoptions(precision=3, suppress=True)

# tokenize and clean the text
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
# tokenize anything that is not a number and not a symbol
word_tokenizer = RegexpTokenizer(r'[^\d\W]+')

nltk.download('stopwords')
nltk.download('wordnet')


sno = SnowballStemmer('english')
wnl = WordNetLemmatizer()

# get our list of stop_words
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
# add some extra stopwords
stop_words |= {"may", "business", "company", "could", "service", "result", "product", 
               "operation", "include", "law", "tax", "change", "financial", "require",
               "cost", "market", "also", "user", "plan", "actual", "cash", "other",
               "thereto", "thereof", "therefore"}

# useful function to print a dictionary sorted by value (largest first by default)
def print_sorted(d, ascending=False):
    factor = 1 if ascending else -1
    sorted_list = sorted(d.items(), key=lambda v: factor*v[1])
    for i, v in sorted_list:
        print("{}: {:.3f}".format(i, v))

# convert text into bag-of-words
def clean_text(txt):
    lemm_txt = [ wnl.lemmatize(wnl.lemmatize(w.lower(),'n'),'v') \
                for w in word_tokenizer.tokenize(txt) if \
                w.isalpha() and w not in stop_words ]
    return [ sno.stem(w) for w in lemm_txt if w not in stop_words and len(w) > 2 ]

corpus = [clean_text(f) for f in files]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id="bag_of_words"></a>

## 1. Bag-of-Words

Implement a function that converts a list of tokenized words into bag-of-words, i.e. a dictionary that outputs the frequency count of a word

** python already provide the `collections.Counter` class to perform this, but I encourage you to implement your own function as an exercise

<a href="#table_of_content">back to top</a>

In [3]:
from collections import defaultdict

def bag_of_words(words):
    '''
    words: a list of words
    '''
    # TO DO
    bag = {}
    for w in words:
        bag[w] = bag.get(w, 0) + 1
            
    return bag


# def bag_of_words(words):
#     bag = defaultdict(int)
#     for w in words:
#         bag[w] += 1
#     return bag

<a id="sentiment"></a>

## 2. Sentiments
Count the fraction of words that appear in a wordlist, returning a sentiment score between 0 and 1:

$$
\textrm{score} = \frac{\textrm{Number of words in document matching wordlist}}{\textrm{Number of words in document}}
$$

Implement the score in a function `get_sentiment(words, wordlist)`, where words is a list of words. Feel free to use the previous `bag_of_words` function. 
(*for extra challenge, try to code the function in one-line*)

Here, I've included a positive and negative wordlist that I constructed by hand. Due to copyright issues, we are not able to provide other commonly used wordlists. I encourage you to try out different wordlists on your own.

<a href="#table_of_content">back to top</a>

In [4]:
# load wordlist first
import pickle

with open('positive_words.pickle', 'rb') as f:
    positive_words = pickle.load(f)
    # also need to stem and lemmatize the text
    positive_words = set(clean_text(" ".join(positive_words)))
    
with open('negative_words.pickle', 'rb') as f:
    negative_words = pickle.load(f)
    negative_words = set(clean_text(" ".join(negative_words)))
    
# check out the list
print("positive words: ", positive_words)
print()
print("negative words: ", negative_words)

positive words:  {'amaz', 'love', 'spring', 'brilliant', 'perfect', 'boon', 'divin', 'success', 'generous', 'resolv', 'eleg', 'favor', 'exquisit', 'refresh', 'boost', 'respect', 'incred', 'solv', 'flourish', 'fabul', 'fascin', 'compliment', 'except', 'abund', 'merit', 'exemplifi', 'devot', 'superb', 'actual', 'prestig', 'win', 'bliss', 'enchant', 'wealthi', 'surg', 'strong', 'clear', 'victori', 'delight', 'strength', 'wonder', 'succeed', 'pleasant', 'glamor', 'inspir', 'bolster', 'power', 'solut', 'talent', 'innov', 'nourish', 'fortun', 'outstand', 'promin', 'appeal', 'distinguish', 'legendari', 'upstand', 'trendi', 'good', 'transform', 'accept', 'exuber', 'lucrat', 'approv', 'rich', 'famous', 'winner', 'applaus', 'benefit', 'motiv', 'benevol', 'marvel', 'reward', 'booster', 'special', 'attract', 'courag', 'admir', 'optim', 'best', 'attain', 'elev', 'optimist', 'independ', 'bless', 'brighter', 'gift', 'great', 'stupend', 'championship', 'honor', 'buoy', 'champion', 'congratul', 'award'

In [5]:
def get_sentiment(txt, wordlist):
    '''
    txt: a list of words
    '''
    # TO DO

    score = len([w for w in txt if w in wordlist]) / len(txt)
    
    return score

In [6]:
# test your function
# Since the there're 11 files in the corpus, we can get a 11-dim array as output
positive_sentiments = np.array([ get_sentiment(c, positive_words) for c in corpus ])
print(positive_sentiments)

negative_sentiments = np.array([ get_sentiment(c, negative_words) for c in corpus ])
print(negative_sentiments)

[ 0.035  0.034  0.031  0.032  0.037  0.031  0.032  0.051  0.039  0.037
  0.037]
[ 0.055  0.06   0.053  0.059  0.058  0.061  0.061  0.063  0.062  0.06
  0.058]


**before continuing part 2, go through the lesson material first!**

<a id="compute_idf"></a>

# Part 2 Document Vector Exercises

## 3. Computer idf
Given a corpus, or a list of bag-of-words, we want to compute for each word $w$, the inverse-document-frequency, or ${\rm idf}(w)$. This can be done in a few steps:

1. Gather a set of all the words in all the bag-of-words (python set comes in handy, and the union operator `|` is useful here)


2. Loop over each word $w$, and compute ${\rm df}_w$, the number of documents where this word appears at least once. A dictionary is useful for keeping track of ${\rm df}_w$


3. After computing ${\rm df}_w$, we can compute ${\rm idf}(w)$. There are usually two possibilities, the simplest one is 
$${\rm idf}(w)=\frac{N}{{\rm df}_w}$$
Where $N$ is the total number of documents in the corpus and $df_w$ the number of documents that contain the word $w$. Frequently, a logarithm term is added as well
$${\rm idf}(w)=\log\frac{N}{{\rm df}_w}$$
One issue with using the logarithm is that when ${\rm df}_w = N$, ${\rm idf}(w)=0$, indicating that words common to all documents would be ignored. If we don't want this behavior, we can define ${\rm idf}(w)=\log\left(1+N/{\rm df}_w\right)$ or ${\rm idf}(w)=1+\log\left(N/{\rm df}_w\right)$ instead. For us, we'll not use the extra +1 for ${\rm idf}$.

In the following, define a function called `get_idf(corpus, include_log=True)` that computes ${\rm idf}(w)$ for all the words in a corpus, where `corpus` for us is a processed list of bag-of-words (stemmed and lemmatized). The optional parameter `include_log` includes the logarithm in the computation.

<a href="#table_of_content">back to top</a>

In [7]:
# compute idf
def get_idf(corpus, include_log=True):
    '''
    corpus: A list of document which is a list of words
    '''
    # TO DO
    n = len(corpus)
    all_words = set()
    for c in corpus:
        all_words |= set(c)
    
    doc_freq = {}
    for w in all_words:
        doc_freq[w] = sum([w in c for c in corpus])
        
    if include_log:
        idf = {w: np.log(n / doc_freq[w]) for w in doc_freq}
    else:
        idf = {w: (n / doc_freq[w]) for w in doc_freq}
        
            
    return idf

You should expect to see many idf values = 0! This is by design, because we have ${\rm idf}(w)=\log N_d/{\rm df}_w$ and $N_d/{\rm df}_w=1$ for the most common words!

In [8]:
# test your code
idf=get_idf(corpus)
print_sorted(idf, ascending=True)

applic: 0.000
manag: 0.000
foreign: 0.000
materi: 0.000
good: 0.000
involv: 0.000
technolog: 0.000
base: 0.000
margin: 0.000
ultim: 0.000
litig: 0.000
particular: 0.000
militari: 0.000
acquir: 0.000
continu: 0.000
way: 0.000
increas: 0.000
delay: 0.000
personnel: 0.000
differ: 0.000
seek: 0.000
anticip: 0.000
often: 0.000
cannot: 0.000
lead: 0.000
associ: 0.000
success: 0.000
retail: 0.000
signific: 0.000
rate: 0.000
event: 0.000
experi: 0.000
depend: 0.000
government: 0.000
spend: 0.000
acquisit: 0.000
due: 0.000
person: 0.000
exchang: 0.000
give: 0.000
avail: 0.000
varieti: 0.000
one: 0.000
abil: 0.000
relat: 0.000
advers: 0.000
general: 0.000
volatil: 0.000
generat: 0.000
declin: 0.000
mani: 0.000
reli: 0.000
industri: 0.000
legal: 0.000
oper: 0.000
subject: 0.000
therebi: 0.000
trade: 0.000
capit: 0.000
high: 0.000
lower: 0.000
suppli: 0.000
reput: 0.000
substanti: 0.000
current: 0.000
extens: 0.000
unabl: 0.000
risk: 0.000
confid: 0.000
power: 0.000
suffer: 0.000
activ: 0.000
prof

<a id="compute_tf"></a>

## 4. Compute tf

Below we will compute ${\rm tf}(w,d)$, or the term frequency for a given word $w$, in a given document $d$. Since our ultimate goal is to compute a document vector, we'd like to keep a few things in mind

1. Store the ${\rm tf}(w, d)$ for each word in a document as a dictionary
2. Even when a word doesn't appear in the document $d$, we still want to keep a $0$ entry in the dictionary. This is important when we convert the dictionary to a vector, where zero entries are important


There are multiple definitions for ${\rm tf}(w,d)$, the simplest one is

$$
{\rm tf}(w,d)=\frac{f_{w,d}}{a_d}
$$

where $f_{w,d}$ is the number of occurence of the word $w$ in the document $d$, and $a$ the average occurence of all the words in that document for normalization. Just like ${\rm idf}(w)$, a logarithm can be added

$$
{\rm tf}(w,d)=\begin{cases}
\frac{1+\log f_{w,d}}{1+\log a_d} & f_{w,d} > 0 \\
0 & f_{w,d} = 0 \\
\end{cases}
$$

Implement the function `get_df(txt, include_log=True)` that computes ${\rm tf}(w,d)$ for each word in the document (returns a defaultdict(int), so that when supplying words not in the document the dictionary will yield zero instead of an error). Also include the optional parameter `include_log` that enables the additional logarithm term in the computation. I suggest also adding a function called `_tf` that computes the function above. 

<a href="#table_of_content">back to top</a>

In [67]:
import numpy as np
from math import *

def _tf(freq, avg, include_log=True):
    # TO DO
    if include_log:
        tf = 0 if freq == 0 else (1 + log(freq)) / (1 + log(avg))
    else:
        tf = freq / avg
        
    return tf


def get_tf(txt, include_log=True):
    '''
    txt: a list of words in a document
    '''
    # TO DO
    freq = bag_of_words(txt)
    avg = np.mean([v for v in freq.values()])
    #avg = np.mean(list(freq.values()))
    tf_dict = {w: _tf(f, avg, include_log) for w, f in freq.items()}
    
    return defaultdict(int, tf_dict)


In [68]:
tfs = [ get_tf(c) for c in corpus ]
print_sorted(tfs[0])

affect: 2.142
oper: 2.113
advers: 2.098
condit: 2.090
materi: 2.082
develop: 1.939
compon: 1.939
price: 1.915
third: 1.891
parti: 1.864
applic: 1.806
certain: 1.806
signific: 1.790
risk: 1.790
new: 1.790
manufactur: 1.757
softwar: 1.757
continu: 1.739
limit: 1.720
futur: 1.701
avail: 1.701
current: 1.680
addit: 1.680
custom: 1.680
subject: 1.680
demand: 1.658
time: 1.658
foreign: 1.658
sale: 1.636
relat: 1.636
effect: 1.612
stock: 1.612
distribut: 1.612
assur: 1.612
consum: 1.586
valu: 1.586
key: 1.586
invest: 1.586
competit: 1.586
technolog: 1.586
compet: 1.586
margin: 1.586
credit: 1.559
devic: 1.559
mani: 1.559
provid: 1.559
resel: 1.559
retail: 1.559
increas: 1.530
mobil: 1.530
communic: 1.530
mac: 1.530
substanti: 1.530
iphon: 1.530
content: 1.530
manag: 1.530
econom: 1.499
depend: 1.499
negat: 1.499
system: 1.499
industri: 1.499
currenc: 1.499
supplier: 1.499
channel: 1.499
comput: 1.499
carrier: 1.499
term: 1.499
obtain: 1.465
person: 1.465
patent: 1.465
network: 1.465
regul: 1.

<a id="doc_vector"></a>

## 5. Document Vector
Combine the implementation for ${\rm tf}(w,d)$ and ${\rm idf}(w)$ to compute a word-vector for each document in a corpus. Don't forget the zero-padding that is needed when a word appears in some document but not others. 

Implmenet the function `get_vectors(tf,idf)`, the input is the output computed by `get_tf` and `get_idf`

(*optional challenge: implement in one line!*)

<a href="#table_of_content">back to top</a>

In [69]:
def get_vector(tf, idf):
    # TO DO
    vec = np.array([tf[w] * idf[w] for w in idf])
    
    return vec

In [70]:
# test your code
doc_vectors = [ get_vector(tf, idf) for tf in tfs ]

for v in doc_vectors:
    print(v)

[ 0.     0.     0.19  ...,  0.281  0.432  0.   ]
[ 0.     0.     0.319 ...,  0.279  0.428  0.   ]
[ 0.     0.537  0.316 ...,  0.314  0.526  0.   ]
[ 0.     0.     0.178 ...,  0.264  0.405  0.   ]
[ 0.36   0.     0.161 ...,  0.238  0.     0.   ]
[ 0.     0.47   0.39  ...,  0.322  0.219  0.483]
[ 0.     0.466  0.386 ...,  0.319  0.217  0.478]
[ 0.  0.  0. ...,  0.  0.  0.]
[ 0.542  0.     0.    ...,  0.     0.     0.422]
[ 0.529  0.     0.    ...,  0.     0.     0.412]
[ 0.461  0.     0.    ...,  0.145  0.     0.753]


<a id ="similarity"></a>

## 6. Similarity

Given two word-vectors $\vec u$ (or $u_i$) and $\vec v$ (or $v_i$), corresponding to two documents, we want to compute different similarity metrics. 

1. Cosine similarity, defined by 
$$
{\rm Sim}_{\cos} = \frac{\vec u \cdot \vec v}{|\vec u| |\vec v|}
$$

2. Jaccard similarity, defined by
$$
{\rm Sim}_{\rm Jaccard} = \frac{\sum_i \min\{u_i, v_i\}}{\sum_i \max\{u_i, v_i\}}
$$

Implement the function similarity as `sim_cis(u,v)` and `sim_jac(u,v)`. Utilize the numpy functions `numpy.linalg.norm` to compute norm of a vector and `np.dot` for computing dot-products. `np.minimum` and `np.maximum` are also useful vectorized pair-wise minimum and maximum functions.

(*optional challenge: implement both functions in one line!*)

<a href="#table_of_content">back to top</a>

In [71]:
from numpy.linalg import norm

def sim_cos(u,v):
    # TO DO
    sim_cos = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    
    return sim_cos


def sim_jac(u,v):
    # TO DO
    sim_jac = np.sum(np.minimum(u, v)) / np.sum(np.maximum(u, v))
    
    return sim_jac

In [72]:
# test your co
# compute all the pairwise similarity metrics
size = len(doc_vectors)
matrix_cos = np.zeros((size,size))
matrix_jac = np.zeros((size,size))

for i in range(size):
    for j in range(size):
        u = doc_vectors[i]
        v = doc_vectors[j]
        matrix_cos[i][j] = sim_cos(u,v)
        matrix_jac[i][j] = sim_jac(u,v)
        
print("Cosine Similarity:")
print(matrix_cos)

print()
print("Jaccard Similarity:")
print(matrix_jac)

Cosine Similarity:
[[ 1.     0.579  0.485  0.375  0.146  0.14   0.143  0.034  0.04   0.032
   0.071]
 [ 0.579  1.     0.812  0.603  0.14   0.173  0.174  0.031  0.035  0.03
   0.075]
 [ 0.485  0.812  1.     0.704  0.136  0.179  0.183  0.032  0.034  0.031
   0.08 ]
 [ 0.375  0.603  0.704  1.     0.14   0.188  0.19   0.049  0.05   0.044
   0.081]
 [ 0.146  0.14   0.136  0.14   1.     0.362  0.367  0.046  0.062  0.056
   0.14 ]
 [ 0.14   0.173  0.179  0.188  0.362  1.     0.962  0.047  0.077  0.063
   0.134]
 [ 0.143  0.174  0.183  0.19   0.367  0.962  1.     0.048  0.078  0.065
   0.136]
 [ 0.034  0.031  0.032  0.049  0.046  0.047  0.048  1.     0.607  0.471
   0.23 ]
 [ 0.04   0.035  0.034  0.05   0.062  0.077  0.078  0.607  1.     0.783
   0.366]
 [ 0.032  0.03   0.031  0.044  0.056  0.063  0.065  0.471  0.783  1.     0.463]
 [ 0.071  0.075  0.08   0.081  0.14   0.134  0.136  0.23   0.366  0.463  1.   ]]

Jaccard Similarity:
[[ 1.     0.493  0.414  0.32   0.109  0.128  0.128  0.032  0.0

### Good Job! You've finished all the exercises!

Here is an optional bonus challenge. We often need to debug other people's code to figure out what's wrong. It's particularly difficult when the code doesn't give errors but computes the wrong quantity. Can you describe why the following implementations for some of the exercises above may be wrong? Highlight the words at the bottom to reveal the solutions!

In [73]:
def get_idf_wrong(corpus, include_log=True):
    freq = defaultdict(int)
    for c in corpus:
        for w in c:
            freq[w] += 1
        
    N = len(corpus)
    if include_log:
        return { w:log(N/freq[w]) for w in freq }
    else:
        return { w:N/freq[w] for w in freq }


def get_sentiment_wrong(txt, wordlist):
    matching_words = [ w for w in wordlist if w in txt ]
    return len(matching_words)/len(txt)

def get_vectors_wrong(tf, idf):
    return np.array([ tf[w]*idf[w] for w in tf ])

# Solutions

Drag your mouse over the white space below this cell, and you'll see details about the solutions.  Or, if it's easier, just double-click on the white space below this cell, which will reveal the cell with hidden text.  Also, please check out the file [bagofwords_solutions.py](bagofwords_solutions.py).

<font color="white">
Solution

get_idf: the defaultdict freq here computes the total number of occurences in all the document. We only want to count it once when a word appears in a document. This may lead to a document frequency larger than N, leading to a negative idf! 

get_sentiment_wrong: if a word w appears twice in the document, it won't be counted properly!

get_vectors_wrong: tf may not contain all the words in idf. We need zero padding for words that appear in idf but not in tf! 
</font> 